In [122]:
#Importación de librearías necesarias
import pandas as pd
from datetime import datetime
import socket
import numpy as np
import math
import time
import pickle  #Para guardar archivos
import os
os.environ['OMP_NUM_THREADS'] = '1'

from pympler import asizeof #Para liberar memoria
import gc

from matplotlib import pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

import scipy.cluster.hierarchy as hierarchy
import scipy.spatial.distance as distance
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage

from sklearn.neighbors import kneighbors_graph
from scipy.interpolate import interp1d
from sklearn.cluster import DBSCAN

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import neighbors

from sklearn.metrics import accuracy_score, precision_score,recall_score,classification_report,confusion_matrix,mean_absolute_error

semilla = 8

In [123]:
#Path general de archivos
if socket.gethostname()=='LAPTOP-PUSGG08B': #Ip de la laptop
    ruta = "E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/Data/TFM/"
    r_ruta = "E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/RData/TFM/"
elif socket.gethostname()=='PCUIOMTDAIE6382': #Ip del working
    ruta =   "D:/Master/Big_Data_Ciencia_Datos/VIU_TFM/Data/TFM/"
    r_ruta = "D:/Master/Big_Data_Ciencia_Datos/VIU_TFM/RData/TFM/"
elif socket.gethostname()=='DESKTOP-11LG84H':
    ruta = "C:/Users/MyHP/Desktop/VIU_TFM"
    r_ruta = "C:/Users/MyHP/Desktop/VIU_TFM"
# Ruta del archivo de pensionistas de vejez
ruta_vj = ruta + 'POB_VEJ_CD656_NEW.dsv'
# Ruta del archivo de historia laboral de pensionistas
ruta_afi = ruta + 'APORTES_CD656_new.dsv'

In [137]:
# Cargar archivo------------------------------------------------------------------------------------------------------------
directorio = r_ruta  
nombre_archivo = 'TABLA_AFI_MAESTRIA.dsv'
ruta_archivo = os.path.join(directorio, nombre_archivo)

data_l = pd.read_csv(ruta_archivo, delimiter=';')  

print(data_l.head())
print(data_l.columns)

   IDENTIFICADOR SEXO FECHA_NACIMIENTO  BASE_CALCULO_2022  IMPO_2022  \
0           1880    H       07/08/1940            29.6081        376   
1           2368    H       08/04/1930             2.8086         77   
2           3174    H       02/02/1948           508.6233        377   
3           4018    H       20/08/1930             0.1746          4   
4           4339    M       16/07/1946           327.7563        139   

   SAL_PROM_1      SUELDO  SAL_PROM_2    Q1_1        Q3_1  ...  N_PRI  N_PUB  \
0    5.602080   2111.9841         NaN  0.0520    3.149800  ...    NaN    NaN   
1    2.161465    170.7557         NaN  0.1600    4.181700  ...    NaN    NaN   
2   93.659464  35403.2774  789.246667  0.4166   60.000000  ...    NaN    NaN   
3    0.174617      1.0477         NaN  0.0725    0.282075  ...    NaN    NaN   
4  138.988210  19875.3140  338.920345  2.6400  334.860000  ...   58.0    NaN   

  N_IND N_VOL_EC  N_VOL_EX  M_PRI  M_PUB  M_IND  M_VOL_EC  M_VOL_EX  
0   NaN      NaN

In [139]:
data_l = data_l.head(500000)

In [140]:
data_l = data_l[~data_l['FECHA_NACIMIENTO'].isin(['01/01/0056', '01/01/0083'])]

In [141]:
# Creación de las variables de los límites superiores
data_l['LS_MS'] = data_l['T3_1'] + 1.5*(data_l['T3_1']-data_l['T1_1'])
data_l['LS1'] = data_l['Q3_1'] + 1.5*(data_l['Q3_1']-data_l['Q1_1'])
data_l['LS2'] = data_l['Q3_2'] + 1.5*(data_l['Q3_2']-data_l['Q1_2'])

In [142]:
# Corrección de variables
data_l= data_l[data_l['SEXO'].notna()]
data_l['BASE_CAL'] = data_l['BASE_CALCULO_2022']
data_l['NUMERO_IMPOSICIONES'] = data_l['IMPO_2022']

In [143]:
# Tranformación de variables
lb = preprocessing.LabelBinarizer()
data_l['SEXO_T'] =data_l['SEXO']
data_l['SEXO'] = lb.fit_transform( data_l['SEXO'])

data_l['RANGO_INI_5MEJ'] = data_l['INICIO_5MEJ']

data_l['RANGO_FIN_5MEJ'] = data_l['FIN_5MEJ']

data_l['LS2'] = np.where( data_l['LS2'].isna(),
                         data_l['LS1'],
                         data_l['LS2'] )

data_l['SAL_PROM2'] = np.where( data_l['SAL_PROM_2'].isna(),
                               data_l['SAL_PROM_1'],
                               data_l['SAL_PROM_2'] )
data_l['PRES'] = 2
data_l['FECHA_NACIMIENTO'] = pd.to_datetime(data_l['FECHA_NACIMIENTO'], format='%d/%m/%Y')
data_l['INICIO_5MEJ'] = pd.to_datetime(data_l['INICIO_5MEJ'], format='%d/%m/%Y')
data_l['FIN_5MEJ'] = pd.to_datetime(data_l['FIN_5MEJ'], format='%d/%m/%Y')

In [144]:
today = datetime.today()

fecha_objetivo = pd.to_datetime('31-12-2023', format='%d-%m-%Y')
data_l['EDAD_J'] = (fecha_objetivo - data_l['FECHA_NACIMIENTO']).dt.days // 365
data_l['TIEM_T'] = data_l['FECHA_NACIMIENTO'].apply(lambda x: today.year - x.year - ((today.month, today.day) < (x.month, x.day)))
data_l['TIEM_MA'] = (data_l['INICIO_5MEJ'] - data_l['FIN_5MEJ']).dt.days // 30
data_l[['M_PRI', 'M_PUB', 'M_IND', 'M_VOL_EC', 'M_VOL_EX',
        'N_PRI', 'N_PUB', 'N_IND', 'N_VOL_EC', 'N_VOL_EX']]= data_l[['M_PRI', 'M_PUB', 'M_IND', 'M_VOL_EC', 'M_VOL_EX','N_PRI', 
                              'N_PUB', 'N_IND', 'N_VOL_EC', 'N_VOL_EX']].fillna(0)
data_l['M_PRI_P'] = data_l['M_PRI'] / data_l[['M_PRI', 'M_PUB', 'M_IND', 'M_VOL_EC', 'M_VOL_EX']].sum(axis=1)
data_l['M_PUB_P'] = data_l['M_PUB'] / data_l[['M_PRI', 'M_PUB', 'M_IND', 'M_VOL_EC', 'M_VOL_EX']].sum(axis=1)
data_l['M_IND_P'] = data_l['M_IND'] / data_l[['M_PRI', 'M_PUB', 'M_IND', 'M_VOL_EC', 'M_VOL_EX']].sum(axis=1)
data_l['M_VOL_EC_P'] = data_l['M_VOL_EC'] / data_l[['M_PRI', 'M_PUB', 'M_IND', 'M_VOL_EC', 'M_VOL_EX']].sum(axis=1)
data_l['M_VOL_EX_P'] = data_l['M_VOL_EX'] / data_l[['M_PRI', 'M_PUB', 'M_IND', 'M_VOL_EC', 'M_VOL_EX']].sum(axis=1)

data_l['N_PRI_P'] = data_l['N_PRI'] / data_l[['N_PRI', 'N_PUB', 'N_IND', 'N_VOL_EC', 'N_VOL_EX']].sum(axis=1)
data_l['N_PUB_P'] = data_l['N_PUB'] / data_l[['N_PRI', 'N_PUB', 'N_IND', 'N_VOL_EC', 'N_VOL_EX']].sum(axis=1)
data_l['N_IND_P'] = data_l['N_IND'] / data_l[['N_PRI', 'N_PUB', 'N_IND', 'N_VOL_EC', 'N_VOL_EX']].sum(axis=1)
data_l['N_VOL_EC_P'] = data_l['N_VOL_EC'] / data_l[['N_PRI', 'N_PUB', 'N_IND', 'N_VOL_EC', 'N_VOL_EX']].sum(axis=1)
data_l['N_VOL_EX_P'] = data_l['N_VOL_EX'] / data_l[['N_PRI', 'N_PUB', 'N_IND', 'N_VOL_EC', 'N_VOL_EX']].sum(axis=1)

In [145]:
import joblib

ruta1 = ruta + '/algoritmos/XGBClassifier_random_search.joblib'
modelo = joblib.load(ruta1)

In [146]:
#Selección de variables para entrenar los modelos
val_otr = ['BASE_CAL', 'LS2', 'SAL_PROM2', 'LS_MS', 'SUELDO',
           'SEXO', 'PRES', 'N_PRI_P', 'N_PUB_P', 'N_IND_P', 'N_VOL_EC_P',
           'N_VOL_EX_P', 'M_PRI_P', 'M_PUB_P', 'M_IND_P', 'M_VOL_EC_P','M_VOL_EX_P',
           'NUMERO_IMPOSICIONES', 'TIEM_T']
data_otr = data_l[ val_otr ]
data_otr

,BASE_CAL,LS2,SAL_PROM2,LS_MS,SUELDO,SEXO,PRES,N_PRI_P,N_PUB_P,N_IND_P,N_VOL_EC_P,N_VOL_EX_P,M_PRI_P,M_PUB_P,M_IND_P,M_VOL_EC_P,M_VOL_EX_P,NUMERO_IMPOSICIONES,TIEM_T
0,29.6081,7.796500,5.602080,65.403050,2111.9841,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,376,84
1,2.8086,10.214250,2.161465,10.646175,170.7557,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77,94
2,508.6233,2378.700000,789.246667,748.000000,35403.2774,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,377,76
3,0.1746,0.596437,0.174617,0.596437,1.0477,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,94
4,327.7563,483.945000,338.920345,483.945000,19875.3140,1,2,0.5,0.0,0.5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,139,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,34.5090,135.542500,77.701108,177.450000,2070.0480,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,84
499996,39.8984,91.933750,77.205400,114.317000,2609.1201,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,152,66
499997,16.9100,16.910000,16.910000,16.910000,16.9100,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,101
499998,149.0880,336.000000,336.000000,406.092000,447.2640,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,98


In [147]:
modelo.predict_proba(data_otr)

array([[9.7307205e-01, 2.6927976e-02],
       [9.6601808e-01, 3.3981942e-02],
       [9.9938297e-01, 6.1699952e-04],
       ...,
       [9.6195811e-01, 3.8041897e-02],
       [9.1126144e-01, 8.8738553e-02],
       [8.1327236e-01, 1.8672767e-01]], dtype=float32)

In [148]:
modelo.predict(data_otr).sum()

136392

In [149]:
136392/500000

0.272784